In [1]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from PIL import Image

In [ ]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection()
cur = conn.cursor()

In [ ]:
cur.execute("""SELECT DISTINCT scene_id
  FROM snapshots \
  WHERE run_id = 6
  ORDER BY created ASC \
  """)

scenes = []
for row in cur:
    res = dict(row)
    scenes.append(res)

print('There are {} scenes'.format(len(scenes)))


### for each scene

In [ ]:
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world
from visualization import load_depth


def scene_to_pointcloud(scene_id):
    conn = visualization.get_connection()
    cur = conn.cursor()
    cur.execute("""SELECT snapshot_id, imagepath, cam_near_clip, camera_fov, width, height, \
      ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as relative_cam_rot, \
      ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos, \
      ARRAY[st_x(camera_rot), st_y(camera_rot), st_z(camera_rot)] as camera_rot\
      FROM snapshots \
      WHERE scene_id = '{}'
      ORDER BY created ASC \
    """.format(scene_id))

    cameras = []
    for row in cur:
        res = dict(row)
        res['camera_rot'] = np.array(res['camera_rot'])
        res['camera_pos'] = np.array(res['camera_pos'])
        res['camera_relative_rotation'] = np.array(res['camera_relative_rotation'])
        cameras.append(res)

    pointclouds = np.empty((3, None))
    for cam in cameras:
        pointcloud = camera_to_pointcloud(cam['imagepath'], cam['cam_near_clip'], cam['camera_fov'])
        pointclouds = np.append(pointclouds, pointcloud)
    return pointclouds
        

def camera_to_pointcloud(name, near_clip, camera_fov, width, height):
    depth = load_depth(name)
    vecs = points_to_homo(res, depth)
    proj_matrix = construct_proj_matrix(height, width, camera_fov, near_clip)
    view_matrix = construct_view_matrix()
    vecs_p = ndc_to_view(vecs, proj_matrix)
    vecs_p_world = view_to_world(vecs_p, view_matrix)
    return vecs_p_world


In [ ]:
workers = 10

run_id = 6

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

conn = visualization.get_connection()
cur = conn.cursor()
cur.execute("""SELECT snapshot_id, imagepath, camera_fov, width, height, cam_near_clip \
    FROM snapshots \
    WHERE run_id = {} \
    ORDER BY snapshot_id DESC \
    """.format(run_id))

results = [dict(res) for res in cur]
#files = [os.path.join(in_directory, i['imagepath']) for i in results]
records = [(os.path.join(in_directory, i['imagepath']), 
            {'H': i['height'], 'W': i['width'], 'fov': i['camera_fov'], 'near_clip': i['cam_near_clip']}) for i in results]


pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(records) * 2).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(tiff_to_dataset_with_transform)
                                              (in_directory, out_directory, name, rgb_format, depth_format, transform)
                         for name, transform in records)
